In [1]:
import pandas as pd
import numpy as np


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df_rating = pd.read_csv("/content/drive/My Drive/Vize_ag/Netflix_Dataset_Rating.csv")
df_movies = pd.read_csv("/content/drive/My Drive/Vize_ag/Netflix_Dataset_Movie.csv")

In [22]:
df_movies.head()

,Movie_ID,Year,Name
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW


In [7]:
df_rating.head()

,User_ID,Rating,Movie_ID
0,712664,5,3
1,1331154,4,3
2,2632461,3,3
3,44937,5,3
4,656399,4,3


In [ ]:
print('movies df dimension:', df_movies.shape, 'rating df dimension:', df_rating.shape)


In [13]:
movies_name = df_movies.set_index('Movie_ID')['Name'].to_dict() # Movie Id ve Movie Name içeren bir Dictionary oluştur
n_users = df_rating['User_ID'].nunique() # user sayısı
n_movies = df_rating['Movie_ID'].nunique() # film sayısı
print('Number of users:', n_users, 'Number of movies:', n_movies)
print('Full rating matrix shape:', n_users * n_movies)
print('number of ratings:', len(df_rating)) # rating sayısı

Number of users: 143458 Number of movies: 1350
Full rating matrix shape: 193668300
number of ratings: 17337458


In [10]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_movies, n_factors=20):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.movie_factors = torch.nn.Embedding(n_movies, n_factors)
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.movie_factors.weight.data.uniform_(0, 0.05)


    def forward(self, user, movie):
        return (self.user_factors(user) * self.movie_factors(movie)).sum(1)



    def predict(self, user, movie):
        return self.forward(user, movie)


In [11]:
# dataloader oluşturuldu (PyTorch için gerekli)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # Verileri makine öğrenimine hazır hale getirmeye yardımcı olan paket


class Loader(Dataset):
    def __init__ (self):
        self.ratings = df_rating.copy()  # dataframe'i kopyala

        users = df_rating.User_ID.unique() # users'ı getir
        movies = df_rating.Movie_ID.unique() # filmleri getir

        self.user2idx = {user: i for i, user in enumerate(users)}  # user id ve index'in dictionary'sini oluştur
        self.movie2idx = {movie: i for i, movie in enumerate(movies)} # movie id ve index'in dictionary'sini oluştur


         # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.user2idx.items()}  # index ve user id'sinin dictionary'sini oluştur
        self.idx2movieid = {i:o for o,i in self.movie2idx.items()}  # index ve movie Id'sinin dictionary'sini oluştur


        self.ratings.Movie_ID = self.ratings.Movie_ID.apply(lambda x: self.movie2idx[x]) # movie Id'yi index'e map'le
        self.ratings.User_ID = self.ratings.User_ID.apply(lambda x: self.user2idx[x]) # user id'yi index'e maple

        self.x = self.ratings.drop('Rating' , axis=1).values
        self.y = self.ratings['Rating'].values
        self.x , self.y = torch.tensor(self.x, dtype=torch.long), torch.tensor(self.y, dtype=torch.float32) # tensora convert et


    def __getitem__(self, index) :
        return (self.x[index], self.y[index])


    def __len__(self):
        return len(self.ratings)   # dataset'in uzunluğunu return et









In [12]:
num_epochs = 128
cuda = torch.cuda.is_available() # cuda'nın varlığını kontrol et
print(cuda)


model = MatrixFactorization(n_users, n_movies, n_factors=20) # model oluştur

for name , param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)   # modelin parametrelerini yazdır

    if cuda:
        model = model.cuda() # model'i gpu'ya taşı


    lose_fn = torch.nn.MSELoss() # kayıp fonksiyonları tanımla



    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) # optimizer'i tanımla


    train_set = Loader()

    train_loader = DataLoader(train_set,128 , shuffle=True) # dataloader oluştur



False
user_factors.weight tensor([[0.0260, 0.0207, 0.0370,  ..., 0.0249, 0.0235, 0.0494],
        [0.0391, 0.0444, 0.0395,  ..., 0.0287, 0.0148, 0.0098],
        [0.0028, 0.0167, 0.0189,  ..., 0.0097, 0.0128, 0.0476],
        ...,
        [0.0320, 0.0330, 0.0273,  ..., 0.0476, 0.0144, 0.0416],
        [0.0107, 0.0181, 0.0445,  ..., 0.0170, 0.0100, 0.0055],
        [0.0404, 0.0377, 0.0071,  ..., 0.0153, 0.0490, 0.0110]])
movie_factors.weight tensor([[0.0086, 0.0034, 0.0245,  ..., 0.0170, 0.0379, 0.0315],
        [0.0249, 0.0222, 0.0228,  ..., 0.0342, 0.0384, 0.0353],
        [0.0371, 0.0369, 0.0164,  ..., 0.0092, 0.0258, 0.0393],
        ...,
        [0.0176, 0.0400, 0.0411,  ..., 0.0235, 0.0115, 0.0494],
        [0.0490, 0.0224, 0.0318,  ..., 0.0401, 0.0395, 0.0420],
        [0.0380, 0.0394, 0.0247,  ..., 0.0141, 0.0307, 0.0231]])


In [14]:
for it in tqdm(range(num_epochs)):
    losses = []         # loss depolamak için list oluştur

for x , y in train_loader:
    if cuda:
        x, y = x.cuda(), y.cuda()
        optimizer.zero_grad()
        output = model(x)
        loss = lose_fn(output.squeeze(), y.type(torch.float64))
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        print('epoch:', it, 'loss:', np.mean(losses))


<ipython-input-14-b51961b30d4d>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

In [15]:
c= 0
a = 0
b = 0
for name , param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

        if c == 0:
           a =param.data
           c += 1
        else:
              b = param.data





user_factors.weight tensor([[0.0260, 0.0207, 0.0370,  ..., 0.0249, 0.0235, 0.0494],
        [0.0391, 0.0444, 0.0395,  ..., 0.0287, 0.0148, 0.0098],
        [0.0028, 0.0167, 0.0189,  ..., 0.0097, 0.0128, 0.0476],
        ...,
        [0.0320, 0.0330, 0.0273,  ..., 0.0476, 0.0144, 0.0416],
        [0.0107, 0.0181, 0.0445,  ..., 0.0170, 0.0100, 0.0055],
        [0.0404, 0.0377, 0.0071,  ..., 0.0153, 0.0490, 0.0110]])
movie_factors.weight tensor([[0.0086, 0.0034, 0.0245,  ..., 0.0170, 0.0379, 0.0315],
        [0.0249, 0.0222, 0.0228,  ..., 0.0342, 0.0384, 0.0353],
        [0.0371, 0.0369, 0.0164,  ..., 0.0092, 0.0258, 0.0393],
        ...,
        [0.0176, 0.0400, 0.0411,  ..., 0.0235, 0.0115, 0.0494],
        [0.0490, 0.0224, 0.0318,  ..., 0.0401, 0.0395, 0.0420],
        [0.0380, 0.0394, 0.0247,  ..., 0.0141, 0.0307, 0.0231]])


In [16]:
trained_movie_factors = model.movie_factors.weight.data.cpu().numpy()

In [17]:
len(trained_movie_factors) # check length


1350

In [19]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10  , random_state=0).fit(trained_movie_factors)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [21]:
for cluster in range(10):
    print('Cluster#{}'.format(cluster))

    movie = []
    for movidx in np.where(kmeans.labels_ == cluster)[0]:
        movid = train_set.idx2movieid[movidx]
        rat_count = df_rating.loc[df_rating['Movie_ID'] == movid].count()[0]
        movie.append((movies_name[movid], rat_count))
    for movie in sorted(movie, key=lambda x: x[1], reverse=True):
        print("\t",movie[0])



Cluster#0
	 Man on Fire
	 The Sum of All Fears
	 Finding Neverland
	 Dogma
	 High Fidelity
	 Beaches
	 The Pianist
	 Jurassic Park III
	 Shanghai Noon
	 Beverly Hills Cop III
	 The Missing
	 Who Framed Roger Rabbit?: Special Edition
	 War Games
	 The King and I
	 The Maltese Falcon
	 The Scorpion King
	 Brother Bear (Theatrical Widescreen Version)
	 Seven Samurai
	 Exit Wounds
	 Like Water for Chocolate
	 Back to the Future Part III
	 Roger & Me
	 Three Men and a Little Lady
	 Bottle Rocket
	 Star Trek: The Motion Picture
	 Rat Race
	 Rising Sun
	 Rambo: First Blood Part II
	 The Bad News Bears
	 For Your Eyes Only
	 The In-Laws
	 Il Postino
	 The Way We Were
	 The Final Cut
	 The Hustler
	 Titan A.E.
	 Joe Versus the Volcano
	 Back to School
	 Annie: Special Anniversary Edition
	 Beyond Borders
	 The Last Temptation of Christ
	 Nothing to Lose
	 Fantasia 2000
	 The Tuskegee Airmen
	 Wolf
	 The Best of Friends: Vol. 4
	 The Dead Zone
	 Three to Tango
	 Rose Red
	 Yellow Submarine
	 All